<a href="https://colab.research.google.com/github/pavelpryadokhin/Stock-forecast/blob/main/Fastapi%2Bstreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Сервис, анализирующий акции и прогнозирующий их цену на следующий день

Создадим сервис, который будет анализировать исторические данные о котировках акций и делать прогнозы на следующий день.

In [ ]:
!pip install streamlit
!pip install fastapi # Установим FastAPI
!pip install uvicorn # Установим ASGI-сервер
!pip install python-multipart # Необходимая зависимость для FastAPI (для работы с данными отправленных форм на сайте)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
%%writefile main.py

from fastapi import FastAPI, Body,UploadFile, File
from fastapi.responses import JSONResponse,PlainTextResponse
import joblib
import keras
from typing import List
import numpy as np
import pandas as pd
from pydantic import BaseModel
import joblib
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization
from keras.layers import Flatten, Conv1D, Conv2D, LSTM, GlobalMaxPooling1D
from keras.models import Sequential, Model


app=FastAPI()
models_scaler= {}
n_input = 50 # Размерность входных данных

def model_predict(scaled_train_data,scaled_test_data,model_id,models_scaler):
    BATCH_SIZE = 20 # Размер пакета

    generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=BATCH_SIZE)
    validator = TimeseriesGenerator(scaled_test_data, scaled_test_data, length=n_input, batch_size=BATCH_SIZE)


    model = Sequential()
    model.add(LSTM(100, input_shape=generator[0][0].shape[1:], return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1,activation="linear"))
    model.compile(optimizer='adam', loss='mse')
    model.summary()

    callbacks = [
    keras.callbacks.ModelCheckpoint(filepath = f'model_{model_id}.keras',
                             monitor = 'val_loss',
                             save_best_only = True,
                             mode = 'min',
                             verbose = 0),]
    history= model.fit(generator, epochs=10, validation_data=validator,callbacks=callbacks)
    model = keras.saving.load_model(f'model_{model_id}.keras')
    print(model_id,models_scaler)
    models_scaler[model_id].append(model)


def predict_price(model,test_x,n_days,n_input):
    predictions = []

    # Первоначальные данные для последующих предсказаний
    current_input = test_x.reshape(-1, 1)

    for _ in range(n_days):
        prediction = model.predict(current_input.reshape(1,n_input,1), verbose=0)
        predictions.append(prediction[0, 0])
        current_input = np.append(current_input[1:], prediction, axis=0)
    return np.array(predictions).reshape(-1, 1)


def rmse_mape(model, scaled_test_data, n_days, scaler,n_input=n_input):
    test=TimeseriesGenerator(scaled_test_data[-(n_input+n_days):], scaled_test_data[-(n_input+n_days):], length=n_input, batch_size=1)
    test_x=test[0][0]

    y_true=scaler.inverse_transform(scaled_test_data[-n_days:])
    y_pred = scaler.inverse_transform(predict_price(model,test_x,n_days,n_input))

    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return rmse,mape


class TrainingData(BaseModel):
    client_id: str
    price: List[List[float]]

class PredictData(BaseModel):
    client_id: str
    price: List[List[float]]
    n_days: int


@app.post('/api/train')
async def train_model(data: TrainingData = Body(...)):
    global models_scaler,n_input
    price = np.array(data.price)

    train_size = int(0.85 * len(price))
    train_data = price[:train_size]
    test_data = price[train_size:]

    scaler = StandardScaler()
    scaler.fit(train_data)
    scaled_train_data = scaler.transform(train_data)
    scaled_test_data = scaler.transform(test_data)

    models_scaler[data.client_id]=[scaler]
    joblib.dump(scaler, f'scaler_{data.client_id}.pkl')

    model_predict(scaled_train_data,scaled_test_data,data.client_id,models_scaler)
    rmse,mape=rmse_mape(models_scaler[data.client_id][1], scaled_test_data, 20, scaler)

    return {'rmse':rmse,'mape':mape}


@app.post('/api/predict')
async def predict_model(data: PredictData = Body(...)):
    global models_scaler,n_input
    try:
        predict_data = np.array(data.price)[-n_input:]
        predict =  models_scaler[data.client_id][0].transform(predict_data).reshape(1,n_input,1)

        result = models_scaler[data.client_id][0].inverse_transform(predict_price(
                            models_scaler[data.client_id][1],predict,data.n_days,n_input))

        return {"predict": result.flatten().tolist()}
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


@app.get("/test")
def get_test():
    return {"Hello": "World"}

In [ ]:
!nohup uvicorn main:app --reload &

nohup: appending output to 'nohup.out'


In [ ]:
!cat nohup.out

In [ ]:
import requests
r = requests.get('http://127.0.0.1:8000/test')
print(r.status_code)
print(r.text)

#если ошибка, то перезагрузите uvicorn и main.py

200
{"Hello":"World"}


In [ ]:
%%writefile streamlit.py
# Импортируем библиотеку
import streamlit as st
import pandas as pd
import requests
import plotly.graph_objs as go
import numpy as np
from datetime import timedelta

st.title("Прогнозирование цен акций с использованием нейронных сетей")

if 'model_trained' not in st.session_state:
    st.session_state.model_trained = False
if 'predictions' not in st.session_state:
    st.session_state.predictions = None

format_map = {
        'ггммдд': '%y%m%d',
        'ддммгг': '%d%m%y',
        'ммддгг': '%m%d%y'
    }


#Вывод графика предсказания
def plot(price,pred_prices):
    price_tail = price.tail(40)
    pred_days=[price_tail.index[-1] + timedelta(days=i) for i in range(1, len(pred_prices) + 1)]
    all_days=price_tail.index.tolist() + pred_days
    all_prices=price_tail.iloc[:, 0].tolist() + pred_prices

    fig = go.Figure()
    # Добавление одной линии для комбинированных цен (фактические + предсказанные)
    fig.add_trace(go.Scatter(x=all_days, y=all_prices,
                            mode='lines+markers',
                            name='Цены',
                            line=dict(color='royalblue', width=2),
                            marker=dict(size=6)))

    # Добавление ограничивающей линии
    fig.add_shape(type='line',
                x0=price_tail.index[-1], x1=price_tail.index[-1],
                y0=0, y1=max(all_prices),
                line=dict(color='red', width=2, dash='dash'),
                name='Начало предсказания')

    # Добавление аннотации для фактических цен
    fig.add_annotation(x=price_tail.index[-1] - pd.Timedelta(days=15),
                    y=max(all_prices) ,text='Фактические цены',
                    font=dict(color='royalblue'))

    fig.add_annotation(x=price_tail.index[-1] + pd.Timedelta(days=1),
                    y=max(all_prices), text='Предсказанные цены',
                    font=dict(color='tomato'))

    fig.update_layout(xaxis_title='Дата', yaxis_title='Цена',
                    legend_title='Тип цен',template='plotly_white',
                    yaxis=dict(range=[min(all_prices) - 10, max(all_prices) + 10]))

    st.plotly_chart(fig)


# Обучения модели
def train(price):
    if 'rmse' not in st.session_state:
        st.session_state.rmse = 0
    if 'mape' not in st.session_state:
        st.session_state.mape = 0

    if not st.session_state.model_trained:
        if st.button("Обучить модель"):
            st.write('Это может занять до 3 минут. Пожалуйста, подождите)')
            response = requests.post(
                "http://127.0.0.1:8000/api/train/",
                json={"client_id": "client_1",'price':np.array(price).tolist()})
            status = response.status_code
            result = response.json()

            rmse= round(result['rmse'],2)
            mape= round(result['mape'],2)
            st.session_state.rmse,st.session_state.mape=rmse,mape
            st.session_state.model_trained = True

    if st.session_state.model_trained:
        st.header("Результаты обучения модели")

        # Вывод RMSE и MAPE с форматированием
        l,r=st.columns(2)
        l.metric("Среднеквадратическая ошибка (RMSE)", f"{st.session_state.rmse:.2f}", delta="")
        r.metric("Средняя абсолютная процентная ошибка (MAPE)", f"{st.session_state.mape:.2f}%", delta="")

        st.write("""
        **Объяснение метрик:**

        - **RMSE** показывает разницу между прогнозируемыми и реальными значениями.
        - **MAPE (%)** измеряет эту разницу относительно реальных значений.
        - Например, значение MAPE 12% указывает на то, что средняя разница между прогнозируемой ценой акции и фактической ценой акции составляет 12%.
        """)
        predict(price)


def predict(price):
    st.header("Прогнозирование модели")

    if st.session_state.model_trained:
        forecast_period = st.slider('Введите срок предсказания (в днях)', 1, 14, 1)

        if st.button("Сделать прогноз"):
            response = requests.post(
                    "http://127.0.0.1:8000/api/predict/",
                    json={"client_id": "client_1",'price':np.array(price).tolist(),'n_days':forecast_period})

            # Сохранение предсказанного периода в сессии + построение графика
            st.session_state.predictions = response.json()['predict']
            plot(price,st.session_state.predictions)






left_column, right_column = st.columns([2,1])

with left_column:
    uploaded_file = st.file_uploader("Загрузите файл с котировками акций", type=["csv"])
with right_column:
    separator = st.radio('Какой разделитель в файле?',['',',',';'])


if uploaded_file and separator:
    df = pd.read_csv(uploaded_file, sep=separator)
    st.write(df.head())

    l,m,r=st.columns([2,2,1])
    with l:
        column_date = st.radio("Выберите столбец, указывающий ДАТУ", options=df.columns.tolist())
    with m:
        column_price = st.radio("Выберите столбец, указывающий ЦЕНУ", options=df.columns.tolist())
    with r:
        format_date = format_map[st.radio("Выберите формат даты", options=['ггммдд','ддммгг','ммддгг'])]


    try:
        price = df[[column_date,column_price]]
        price[column_date] = pd.to_datetime(price[column_date], format=format_date)
        price.set_index(column_date, inplace=True)
        col1,col2=st.columns([1,2])
        col1.write(price.head())
        col2.area_chart(price)
        if "confirmed" not in st.session_state:
            if st.button("Подтвердить выбор"):
                st.session_state.confirmed = True  # Сохраняем состояние

    except:
        st.write('Ошибка. Попробуйте изменить данные в столбцах')

    if  "confirmed" in st.session_state:
        train(price)






Writing streamlit.py


In [ ]:
!streamlit run streamlit.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

ssh: connect to host serveo.net port 22: Connection refused


#Пути улучшения

В будущем можно усовершенствовать процесс прогнозирования, сделав модель более сложной. Также стоит рассмотреть возможность включения в процесс принятия решений модели BERT, которая способна анализировать новости и отражать общее настроение рынка.